<a href="https://colab.research.google.com/github/thandongdatviet/Code-classifier/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install pandas


##1. Xử lí dữ liệu

In [2]:
import pandas as pd
import json

# Hàm đọc dữ liệu từ tệp txt
def read_data(file_path):
    df = pd.read_csv(file_path, sep='\t', header=None, names=['code1_id', 'code2_id', 'label'])
    return df


# Đọc các tệp dữ liệu
train_df = read_data('train_100.txt')
test_df = read_data('test_100.txt')
valid_df = read_data('valid_100.txt')

print("\nTập Train:")
print(train_df.head())

print("\nTập Test:")
print(test_df.head())

print("\nn Tập Validate:")
print(valid_df.head())



Tập Train:
   code1_id  code2_id  label
0  11314030  13307307      2
1   1798720  17871427      3
2    520600    800221      0
3    124387    881838      0
4      9563    420619      0

Tập Test:
   code1_id  code2_id  label
0   2668636  16408803      3
1    411631   1421557      1
2  16209101  17420328      1
3    211685    689532      0
4     47337  10798829      2

n Tập Validate:
   code1_id  code2_id  label
0    831649   6814855      1
1    536614  21288637      3
2     53722    718537      0
3   2711644  21488869      1
4    333457    665895      0


In [3]:
# Hàm đọc data.jsonl
def read_jsonl(file_path):
    data = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            obj = json.loads(line)
            idx = obj['idx']
            func = obj['func']
            data[idx] = func
    return data

# Đọc data.jsonl
code_dict = read_jsonl('data.jsonl')

print(f"Tổng số đoạn mã: {len(code_dict)}")
print("Ví dụ về một đoạn mã:")
sample_idx = list(code_dict.keys())[0]
print(f"ID: {sample_idx}")
print(code_dict[sample_idx])



Tổng số đoạn mã: 9126
Ví dụ về một đoạn mã:
ID: 10000832
    public static void main(String[] args) {
        LogFrame.getInstance();
        for (int i = 0; i < args.length; i++) {
            String arg = args[i];
            if (arg.trim().startsWith(DEBUG_PARAMETER_NAME + "=")) {
                properties.put(DEBUG_PARAMETER_NAME, arg.trim().substring(DEBUG_PARAMETER_NAME.length() + 1).trim());
                if (properties.getProperty(DEBUG_PARAMETER_NAME).toLowerCase().equals(DEBUG_TRUE)) {
                    DEBUG = true;
                }
            } else if (arg.trim().startsWith(MODE_PARAMETER_NAME + "=")) {
                properties.put(MODE_PARAMETER_NAME, arg.trim().substring(MODE_PARAMETER_NAME.length() + 1).trim());
            } else if (arg.trim().startsWith(AUTOCONNECT_PARAMETER_NAME + "=")) {
                properties.put(AUTOCONNECT_PARAMETER_NAME, arg.trim().substring(AUTOCONNECT_PARAMETER_NAME.length() + 1).trim());
            } else if (arg.trim().startsW

In [4]:
# Hàm kết hợp hai đoạn mã thành một chuỗi
def combine_codes(code1_id, code2_id):
    code1 = code_dict.get(str(code1_id), "")
    code2 = code_dict.get(str(code2_id), "")
    return f"{code1} [SEP] {code2}"

# Áp dụng kết hợp mã vào các DataFrame
train_df['combined_code'] = train_df.apply(lambda row: combine_codes(row['code1_id'], row['code2_id']), axis=1)
test_df['combined_code'] = test_df.apply(lambda row: combine_codes(row['code1_id'], row['code2_id']), axis=1)
valid_df['combined_code'] = valid_df.apply(lambda row: combine_codes(row['code1_id'], row['code2_id']), axis=1)

print(train_df[['code1_id', 'code2_id', 'combined_code', 'label']].head())


   code1_id  code2_id                                      combined_code  \
0  11314030  13307307      public static void copyFile(File source, F...   
1   1798720  17871427      public static void save(String packageName...   
2    520600    800221      public void convert(File src, File dest) t...   
3    124387    881838                                             [SEP]    
4      9563    420619      public void convert(File src, File dest) t...   

   label  
0      2  
1      3  
2      0  
3      0  
4      0  


In [5]:
# Kiểm tra phân bố nhãn
print("Phân bố nhãn trong tập huấn luyện:")
print(train_df['label'].value_counts())


Phân bố nhãn trong tập huấn luyện:
label
2    15
3    15
0    15
1    15
Name: count, dtype: int64


##2. Triển khai mô hình BERT

###2.1. Mã hóa với BERT tokenizer

In [11]:
import torch
from transformers import BertTokenizer

# Sử dụng tokenizer của BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Hàm mã hóa dữ liệu
def encode_data_bert(texts, labels, max_len=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer.encode_plus(
            text,
            add_special_tokens=True,      # Thêm [CLS] và [SEP]
            max_length=max_len,          # Cắt hoặc padding về max_len
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'           # Trả về tensor
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    # Chuyển thành tensor
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.values)

    return input_ids, attention_masks, labels

train_inputs_bert, train_masks_bert, train_labels_bert = encode_data_bert(train_df['combined_code'], train_df['label'])
test_inputs_bert, test_masks_bert, test_labels_bert = encode_data_bert(test_df['combined_code'], test_df['label'])
valid_inputs_bert, valid_masks_bert, valid_labels_bert = encode_data_bert(valid_df['combined_code'], valid_df['label'])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Tạo TensorDatasets
train_dataset_bert = TensorDataset(train_inputs_bert, train_masks_bert, train_labels_bert)
test_dataset_bert = TensorDataset(test_inputs_bert, test_masks_bert, test_labels_bert)
valid_dataset_bert = TensorDataset(valid_inputs_bert, valid_masks_bert, valid_labels_bert)

# Tạo DataLoaders
batch_size =32

train_dataloader_bert = DataLoader(
    train_dataset_bert,
    sampler=RandomSampler(train_dataset_bert),
    batch_size=batch_size
)

test_dataloader_bert = DataLoader(
    test_dataset_bert,
    sampler=SequentialSampler(test_dataset_bert),
    batch_size=batch_size
)

valid_dataloader_bert = DataLoader(
    valid_dataset_bert,
    sampler=SequentialSampler(valid_dataset_bert),
    batch_size=batch_size
)


In [12]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

# Tải mô hình BERT với đầu ra phân loại
model_bert = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=4,              # 4 lớp: 0,1,2,3
    output_attentions=False,
    output_hidden_states=False
)

# Sử dụng GPU nếu có
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_bert.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [14]:
# Định nghĩa optimizer
optimizer_bert = AdamW(model_bert.parameters(),
                      lr=2e-5,
                      eps=1e-8)

# Định nghĩa số epoch
epochs = 10

# Tính tổng số bước
total_steps_bert = len(train_dataloader_bert) * epochs
print(total_steps_bert)

# Tạo scheduler
scheduler_bert = get_linear_schedule_with_warmup(optimizer_bert,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps_bert)


20


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

def train_epoch(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0

    for step, batch in enumerate(dataloader):
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        model.zero_grad()

        outputs = model(
            input_ids=b_input_ids,
            attention_mask=b_attention_mask,
            labels=b_labels
        )

        loss = outputs.loss
        # print(f"Loss: {loss}")
        total_loss += loss.item()

        loss.backward()

        # Cắt gradient để tránh hiện tượng exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(dataloader)

    print(f"Average training loss: {avg_train_loss}")

def evaluate(model, dataloader):
    model.eval()
    preds = []
    true_labels = []

    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask
            )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).flatten()

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    print(f"Validation Accuracy: {acc}")
    print("Classification Report:")
    print(classification_report(true_labels, preds, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3']))


In [ ]:
for epoch in range(1, epochs + 1):
    print(f"\n======== Epoch {epoch} / {epochs} ========")
    train_epoch(model_bert, train_dataloader_bert, optimizer_bert, scheduler_bert)
    evaluate(model_bert, valid_dataloader_bert)




```
# This is formatted as code
```

##3. CodeBERT

###3.1. Mã hóa với CodeBERT tokenizer

In [9]:
from transformers import RobertaTokenizer
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score, classification_report

# Tải tokenizer của CodeBERT
tokenizer_codebert = RobertaTokenizer.from_pretrained("microsoft/codebert-base")

# Hàm mã hóa dữ liệu cho CodeBERT
def encode_data_codebert(texts, labels, max_len=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer_codebert.encode_plus(
            text,
            add_special_tokens=True,      # Thêm <s> và </s> cho RoBERTa
            max_length=max_len,          # Cắt hoặc padding về max_len
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'           # Trả về tensor
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    # Chuyển thành tensor
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.values)

    return input_ids, attention_masks, labels

# Mã hóa dữ liệu huấn luyện, kiểm tra và xác thực cho CodeBERT
train_inputs_codebert, train_masks_codebert, train_labels_codebert = encode_data_codebert(train_df['combined_code'], train_df['label'])
test_inputs_codebert, test_masks_codebert, test_labels_codebert = encode_data_codebert(test_df['combined_code'], test_df['label'])
valid_inputs_codebert, valid_masks_codebert, valid_labels_codebert = encode_data_codebert(valid_df['combined_code'], valid_df['label'])



In [24]:
print(test_labels_codebert)

tensor([3, 1, 1, 0, 2, 2, 3, 3, 0, 0, 1, 3, 1, 2, 2, 0, 0, 1, 3, 2])


###3.2. Tạo dataLoader

In [16]:
# Tạo TensorDatasets
train_dataset_codebert = TensorDataset(train_inputs_codebert, train_masks_codebert, train_labels_codebert)
test_dataset_codebert = TensorDataset(test_inputs_codebert, test_masks_codebert, test_labels_codebert)
valid_dataset_codebert = TensorDataset(valid_inputs_codebert, valid_masks_codebert, valid_labels_codebert)

# Tạo DataLoaders
train_dataloader_codebert = DataLoader(
    train_dataset_codebert,
    sampler=RandomSampler(train_dataset_codebert),
    batch_size=batch_size
)

test_dataloader_codebert = DataLoader(
    test_dataset_codebert,
    sampler=SequentialSampler(test_dataset_codebert),
    batch_size=batch_size
)

valid_dataloader_codebert = DataLoader(
    valid_dataset_codebert,
    sampler=SequentialSampler(valid_dataset_codebert),
    batch_size=batch_size
)


###3.3. Khỏie tạo CodeBERT

In [19]:
from transformers import RobertaForSequenceClassification, AdamW, get_linear_schedule_with_warmup,RobertaModel

# Tải mô hình CodeBERT với đầu ra phân loại
model_codebert = RobertaForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=4,  # 4 lớp: 0,1,2,3
    output_attentions=False,
    output_hidden_states=False
)

# Sử dụng GPU nếu có
model_codebert.to(device)


pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

###3.4. Khởi tạo optimizer

In [20]:
# Định nghĩa optimizer
optimizer_codebert = AdamW(model_codebert.parameters(),
                           lr=2e-5,
                           eps=1e-8)

# Định nghĩa số epoch
epochs = 10

# Tính tổng số bước
total_steps_codebert = len(train_dataloader_codebert) * epochs

# Tạo scheduler
scheduler_codebert = get_linear_schedule_with_warmup(optimizer_codebert,
                                                     num_warmup_steps=0,
                                                     num_training_steps=total_steps_codebert)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


###3.5. Định nghĩa hàm train và evaluate

In [21]:
# Hàm huấn luyện cho CodeBERT
def train_epoch_codebert(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0

    for step, batch in enumerate(dataloader):
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        model.zero_grad()

        outputs = model(
            input_ids=b_input_ids,
            attention_mask=b_attention_mask,
            labels=b_labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()

        # Cắt gradient để tránh hiện tượng exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(dataloader)
    print(f"Average training loss: {avg_train_loss}")

# Hàm đánh giá cho CodeBERT
def evaluate_codebert(model, dataloader):
    model.eval()
    preds = []
    true_labels = []

    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask
            )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).flatten()

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    print(f"Validation Accuracy: {acc}")
    print("Classification Report:")
    print(classification_report(true_labels, preds, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3']))


###3.6. Training

In [ ]:
for epoch in range(1, epochs + 1):
    print(f"\n======== Epoch {epoch} / {epochs} ========")
    train_epoch_codebert(model_codebert, train_dataloader_codebert, optimizer_codebert, scheduler_codebert)
    evaluate_codebert(model_codebert, valid_dataloader_codebert)


##4. CodeT5+

###4.1.

In [ ]:
from transformers import AutoTokenizer

# Tải tokenizer của CodeT5
tokenizer_codet5 = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

# Hàm mã hóa dữ liệu cho CodeT5
def encode_data_codet5(texts, labels, max_len=256):
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded = tokenizer_codet5.encode_plus(
            text,
            add_special_tokens=True,      # Thêm các token đặc biệt
            max_length=max_len,          # Cắt hoặc padding về max_len
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'           # Trả về tensor
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])

    # Chuyển thành tensor
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels.values)

    return input_ids, attention_masks, labels

# Mã hóa dữ liệu huấn luyện, kiểm tra và xác thực cho CodeT5
train_inputs_codet5, train_masks_codet5, train_labels_codet5 = encode_data_codet5(train_df['combined_code'], train_df['label'])
test_inputs_codet5, test_masks_codet5, test_labels_codet5 = encode_data_codet5(test_df['combined_code'], test_df['label'])
valid_inputs_codet5, valid_masks_codet5, valid_labels_codet5 = encode_data_codet5(valid_df['combined_code'], valid_df['label'])


###4.2.

In [ ]:
# Tạo TensorDatasets
train_dataset_codet5 = TensorDataset(train_inputs_codet5, train_masks_codet5, train_labels_codet5)
test_dataset_codet5 = TensorDataset(test_inputs_codet5, test_masks_codet5, test_labels_codet5)
valid_dataset_codet5 = TensorDataset(valid_inputs_codet5, valid_masks_codet5, valid_labels_codet5)

# Tạo DataLoaders
train_dataloader_codet5 = DataLoader(
    train_dataset_codet5,
    sampler=RandomSampler(train_dataset_codet5),
    batch_size=batch_size
)

test_dataloader_codet5 = DataLoader(
    test_dataset_codet5,
    sampler=SequentialSampler(test_dataset_codet5),
    batch_size=batch_size
)

valid_dataloader_codet5 = DataLoader(
    valid_dataset_codet5,
    sampler=SequentialSampler(valid_dataset_codet5),
    batch_size=batch_size
)


###4.3.

In [ ]:
from transformers import AutoModelForSequenceClassification

# Tải mô hình CodeT5 với đầu ra phân loại
model_codet5 = AutoModelForSequenceClassification.from_pretrained(
    "Salesforce/codet5-base",
    num_labels=4,  # 4 lớp: 0,1,2,3
    output_attentions=False,
    output_hidden_states=False
)

# Sử dụng GPU nếu có
model_codet5.to(device)


###4.4.

In [ ]:
# Định nghĩa optimizer
optimizer_codet5 = AdamW(model_codet5.parameters(),
                         lr=2e-5,
                         eps=1e-8)

# Định nghĩa số epoch
epochs = 10

# Tính tổng số bước
total_steps_codet5 = len(train_dataloader_codet5) * epochs

# Tạo scheduler
scheduler_codet5 = get_linear_schedule_with_warmup(optimizer_codet5,
                                                   num_warmup_steps=0,
                                                   num_training_steps=total_steps_codet5)



###4.5.

In [ ]:
# Hàm huấn luyện cho CodeT5
def train_epoch_codet5(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0

    for step, batch in enumerate(dataloader):
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        model.zero_grad()

        outputs = model(
            input_ids=b_input_ids,
            attention_mask=b_attention_mask,
            labels=b_labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()

        # Cắt gradient để tránh hiện tượng exploding gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(dataloader)
    print(f"Average training loss: {avg_train_loss}")

# Hàm đánh giá cho CodeT5
def evaluate_codet5(model, dataloader):
    model.eval()
    preds = []
    true_labels = []

    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask
            )

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).flatten()

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    print(f"Validation Accuracy: {acc}")
    print("Classification Report:")
    print(classification_report(true_labels, preds, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3']))


###4.6.

In [ ]:
for epoch in range(1, epochs + 1):
    print(f"\n======== Epoch {epoch} / {epochs} ========")
    train_epoch_codet5(model_codet5, train_dataloader_codet5, optimizer_codet5, scheduler_codet5)
    evaluate_codet5(model_codet5, valid_dataloader_codet5)


##5. Đánh giá các model

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score

def detailed_evaluation(model, dataloader, model_name="Model"):
    model.eval()
    preds = []
    true_labels = []
    probs = []

    for batch in dataloader:
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(
                input_ids=b_input_ids,
                attention_mask=b_attention_mask
            )

        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=1)
        predictions = torch.argmax(probabilities, dim=1).flatten()

        preds.extend(predictions.cpu().numpy())
        true_labels.extend(b_labels.cpu().numpy())
        probs.extend(probabilities.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    print(f"{model_name} - Accuracy: {acc}")
    print(f"{model_name} - Classification Report:")
    print(classification_report(true_labels, preds, target_names=['Class 0', 'Class 1', 'Class 2', 'Class 3']))

    # Confusion Matrix
    cm = confusion_matrix(true_labels, preds)
    print(f"{model_name} - Confusion Matrix:")
    print(cm)


In [ ]:
# BERT
detailed_evaluation(model_bert, test_dataloader_bert, model_name="BERT")

# CodeBERT
detailed_evaluation(model_codebert, test_dataloader_codebert, model_name="CodeBERT")

# CodeT5+
detailed_evaluation(model_codet5, test_dataloader_codet5, model_name="CodeT5+")


tăng số epoch 100-200
thay đổi tham sô nếu không hiệu quả
sửa lại đầu vào input
thử với graph neural network
chạy hết 100000 mẫu
